In [ ]:
#@title
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import keras
import os
import random
import tensorflow as tf

from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, KFold
from keras.regularizers import l1, l2, l1_l2
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Normalization
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import LeakyReLU, ReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.framework import ops

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
ccdb = pd.read_csv('drive/MyDrive/concrete.csv')
ccdb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
# here to avoid errors, make sure x and y are just numpy arrays (extract the values from the data frames)
x = ccdb.loc[:,'cement':'age'].values
y = ccdb["strength"].values

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
scale = StandardScaler()

In [ ]:
def make_model(lr=0.001):
  model= Sequential()
  model.add(Dense(8, use_bias=True, activation="gelu", input_dim=8))
  model.add(Dense(4, activation="gelu"))
  model.add(Dense(1, activation="linear"))
  model.summary()
  opt=Adam(learning_rate=lr)
  model.compile(loss="mse", optimizer=opt)
  return model

MSE_list=[]
for idxtrain, idxtest in kf.split(x):
  model = make_model(lr=0.001)
  xtrain = scale.fit_transform(x[idxtrain])
  xtest = scale.transform(x[idxtest])
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  model.fit(xtrain, ytrain, epochs=2000, verbose=0)
  MSE_list.append(mse(ytest, model.predict(xtest)))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
7/7 [==============================] - 0s 1ms/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 72        
           

In [ ]:
MSE_list

[33.55940141369618,
 28.811494581396094,
 29.427577688333063,
 45.11166339182459,
 38.238587422686585]

In [ ]:
MSE_mean=np.mean(MSE_list)
print(str(MSE_mean))

35.0297448995873
